In [1]:
# Hacer peticion HTTP
import requests
# Manipular código y guardar datos tabulares en archivo CSV
import pandas as pd

In [2]:
# url de la página web a «escrapear»
url = 'https://en.wikipedia.org/wiki/List_of_supermarket_chains'

In [4]:
# pasar "User-agent" para simular interacción con la página usando Navegador web
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

respuesta = requests.get(url, headers=headers)

# El código de respuesta <200> indicará que todo salió bien
print(respuesta)


<Response [200]>


In [5]:
all_tables = pd.read_html(respuesta.content, encoding = 'utf8')

In [6]:
print(f'Total de tablas encontradas: {len(all_tables)}')

Total de tablas encontradas: 3


In [7]:
matched_table = pd.read_html(respuesta.text, match='Ahold Delhaize')

# imprime numero de tablas que coinciden con parametro match
print(f'Total de tablas encontradas: {len(matched_table)}')

Total de tablas encontradas: 1


<ipython-input-7-240112dbdfbf>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matched_table = pd.read_html(respuesta.text, match='Ahold Delhaize')


In [11]:
# Guardar tabla en variable con nombre semántico
multinational = matched_table[0]

# Verificamos si es la tabla que buscamos
multinational.tail(10)

,Company,Headquarters,Served countries (besides the headquarters),Map,Number of locations,Number of employees,Unnamed: 6
364,Kansai Super,Japan,NaN,NaN,60,NaN,NaN
365,Extra Foods,Canada,NaN,NaN,4,NaN,NaN
366,Globus Department Store,Switzerland,NaN,NaN,NaN,3400,NaN
367,Globus,Germany,"Czech Republic, Russia, Luxembourg",NaN,165,44900,NaN
368,Allied Stores,United States,NaN,NaN,226,NaN,NaN
369,T&T Supermarket,Canada,NaN,NaN,33,NaN,NaN
370,Match,Belgium,"Luxembourg, France",NaN,217,NaN,NaN
371,C-market,Serbia,NaN,NaN,NaN,NaN,NaN
372,Tegut,Germany,NaN,NaN,275,7700,NaN
373,Comet,United Kingdom,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Solo si tenemos valores que no van en la tabla y estan en las dos últimas filas
# Remover ultima(s) n fila(s)
multinational.drop(multinational.tail(2).index, inplace=True)

# Verificar si se eliminaron los registros no deseados
multinational.tail(2)

In [17]:
# Crear una columna 'row_id' con valores secuenciales
multinational['row_id'] = range(1, len(multinational) + 1)

# Luego, configurar 'row_id' como el índice
multinational.set_index('row_id', inplace=True)

# Verificar el cambio de índice
multinational.head(10)



,Company,Headquarters,Served countries (besides the headquarters),Map,Number of locations,Number of employees,Unnamed: 6
row_id,,,,,,,
1,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",NaN,84500,170000,NaN
2,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",NaN,20008,"560,000+",NaN
3,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",NaN,7659,375000,NaN
4,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",NaN,5241,72811,NaN
5,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",NaN,7178,201361,NaN
6,Aldi,Germany (Süd and Nord),"Australia (Süd), Austria (Süd as Hofer), China...",NaN,"12,596 (13,153 if including Trader Joe's)",274172,NaN
7,Edeka,Germany,Denmark,NaN,13646,381000,NaN
8,Extra,Germany,NaN,NaN,NaN,9700,NaN
9,Tengelmann Group,Germany,NaN,NaN,4170,72714,NaN


In [18]:
from unicodedata import normalize

In [19]:
def remove_whitespace(x):
    """Funcion para normalizar datos con Unicode para luego quitar los espacios usando .replace().

    Argumentos de entrada: Nombre de columna o lista con nombres de columnas.
    Retorna: columna o columnas sin espacios en blanco
    """
    if isinstance(x, str):
        return normalize('NFKC', x).replace(' ', '')
    else:
        return x

In [20]:
# Guardamos en variable nombre de columnas a quitar espacios en blanco
numeric_cols = ['Map','Number of locations','Number of employees', 'Unnamed: 6']

# Aplicar función remove_whitespace a columnas en variable y las reemplazamos en tabla
multinational[numeric_cols] = multinational[numeric_cols].applymap(remove_whitespace)

# Verificamos si se quitaron los espacios en blanco
multinational.head()

<ipython-input-20-f7261b16f325>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  multinational[numeric_cols] = multinational[numeric_cols].applymap(remove_whitespace)


,Company,Headquarters,Served countries (besides the headquarters),Map,Number of locations,Number of employees,Unnamed: 6
row_id,,,,,,,
1,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",NaN,84500,170000,NaN
2,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",NaN,20008,"560,000+",NaN
3,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",NaN,7659,375000,NaN
4,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",NaN,5241,72811,NaN
5,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",NaN,7178,201361,NaN


In [21]:
# Mostrar tipo de datos de la tabla
multinational.dtypes


,0
Company,object
Headquarters,object
Served countries (besides the headquarters),object
Map,float64
Number of locations,object
Number of employees,object
Unnamed: 6,float64


In [28]:
import re
import pandas as pd

# Función para limpiar los datos y extraer solo los números
def clean_number(value):
    # Si el valor es NaN o ya es numérico, lo devolvemos sin cambios
    if pd.isna(value) or isinstance(value, (int, float)):
        return value
    # Convertir a cadena si es necesario
    value = str(value)
    # Reemplazar el texto dentro de paréntesis, comas y eliminar signos como '+'
    value = re.sub(r'\([^)]*\)', '', value)  # Elimina el texto dentro de paréntesis
    value = value.replace(',', '')  # Elimina las comas
    value = value.replace('+', '')  # Elimina el signo '+'
    try:
        return float(value)  # Convertir el valor limpio a float
    except ValueError:
        return None  # Retorna None si no puede convertir

# Aplicar la función de limpieza a las columnas 'Number of locations' y 'Number of employees'
multinational['Number of locations'] = multinational['Number of locations'].apply(clean_number)
multinational['Number of employees'] = multinational['Number of employees'].apply(clean_number)

# Ahora puedes aplicar la conversión de tipos
convert_dict = {
    'Company': 'string',
    'Headquarters': 'string',
    'Served countries (besides the headquarters)': 'string',
    'Map': 'string',
    'Number of locations': 'float64',
    'Number of employees': 'float64',
    'Unnamed: 6': 'float64'
}

# Aplicar la conversión de tipos
multinational = multinational.astype(convert_dict)

# Verificamos los tipos de datos
print(multinational.dtypes)



Company                                        string[python]
Headquarters                                   string[python]
Served countries (besides the headquarters)    string[python]
Map                                            string[python]
Number of locations                                   float64
Number of employees                                   float64
Unnamed: 6                                            float64
dtype: object


In [29]:
# Guarda Dataframe a archivo CSV
multinational.to_csv('multinational.csv')



In [30]:
# Leamos el archivo para verificar su creacion
pd.read_csv('multinational.csv').head(10)

,row_id,Company,Headquarters,Served countries (besides the headquarters),Map,Number of locations,Number of employees,Unnamed: 6
0,1,7-Eleven,Japan United States,"Australia, Canada, China, Cambodia, Denmark, H...",NaN,84500.0,170000.0,NaN
1,2,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",NaN,20008.0,560000.0,NaN
2,3,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",NaN,7659.0,375000.0,NaN
3,4,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",NaN,5241.0,72811.0,NaN
4,5,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",NaN,7178.0,201361.0,NaN
5,6,Aldi,Germany (Süd and Nord),"Australia (Süd), Austria (Süd as Hofer), China...",NaN,12596.0,274172.0,NaN
6,7,Edeka,Germany,Denmark,NaN,13646.0,381000.0,NaN
7,8,Extra,Germany,NaN,NaN,NaN,9700.0,NaN
8,9,Tengelmann Group,Germany,NaN,NaN,4170.0,72714.0,NaN
9,10,Minimal,Germany,Poland,NaN,1500.0,NaN,NaN


In [31]:
# Cargar o descargar archivos
from google.colab import files



In [32]:
# Descarga archivo con datos de tabla
files.download("multinational.csv")

print('Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...
